1 - Provide Path to your API Key

In [ ]:
openai_key_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/openai_key.txt'

2 - Preprocess the Abstracts
- Provide the path to the location you saved the .txt file with the abstracts

In [ ]:
# Define your path to the text file with the abstracts in it
abstracts_txt_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/review_pyper/raws/abstract_results.txt'

In [ ]:
from calvin_utils.gpt_sys_review.txt_utils import AbstractSeparator
separator = AbstractSeparator(abstracts_txt_path)
df, abstracts_txt_path_preprocessed = separator.run()
df

3 - Shorten List of Abstracts to Those Passing Title Screen

In [ ]:
# Define your path to the CSV which contains the successfully screened titles
title_review_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/review_pyper/raws/pubmed_results_cleaned.csv"

In [ ]:
from calvin_utils.gpt_sys_review.txt_utils import TitleReviewFilter
title_review_filter = TitleReviewFilter(title_review_path, abstracts_path=abstracts_txt_path_preprocessed, column_name="OpenAI_Screen")
df, abstracts_txt_path_preprocessed_filtered = title_review_filter.run()
df

4 - Evaluate Abstracts That Passed Title Screen
- Define your question, like in this example. Make it a question you WANT a yes to.     
      - question =  "Do you think this article may contain a case report of amnesia?"


In [ ]:
question = "Do you think this article may contain a case report of amnesia?"

In [ ]:
from calvin_utils.gpt_sys_review.gpt_utils import AbstractScreener
abstract_screening = AbstractScreener(api_key_path=openai_key_path, csv_path=abstracts_txt_path_preprocessed_filtered, question=question, model_choice="gpt4", keywords=["amnesia", "memory", "amnestic", "forget"])
screened_abstract_path = abstract_screening.run()

5 - Postprocess Results

In [ ]:
from calvin_utils.gpt_sys_review.txt_utils import PostProcessing
post_processor = PostProcessing(file1_path=abstracts_txt_path_preprocessed, file2_path=screened_abstract_path, pubmed_csv_path=title_review_path)
df = post_processor.run()
df.head(2)

Your abstracts have now been screened. 
- If you are curious about screening titles then abstracts versus titles and abstracts, please see this study: 
    - doi: 10.2147/CLEP.S43118
- Enjoy. If this has been helpful, please consider adding Calvin Howard as a collaborator. 
- e: choward12@bwh.harvard.edu